<div dir=rtl>
    
# تکلیف دوم عملی

## فرزانه کوهستانی - 40115824

</div>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import os
from os import walk
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
import torch.utils.data as data
import matplotlib.pyplot as plt
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from sklearn import metrics
import warnings
import time 
import tqdm
from tqdm import trange 
from tqdm import tqdm
import cv2
import sys
from google.colab.patches import cv2_imshow
from PIL import Image
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'google.colab'

<div dir=rtl>

# Question 1
    
</div>

<div dir=rtl>
در سلول بالا تمام ماژولهای لازم برای انجام پروژه را اضافه کرده ام.

من برای پیاده سازی این تکلیف از فریمورک پایتورچ استفاده نموده ام.
</div>

In [ ]:
# PATH = "C:\\uni\\1 - first term\\Deep Learning\\HW2 Practical"
# data_dir = PATH + '\\train' # load from Kaggle
# data_dir = '/content/drive/MyDrive/data'
transform = transforms.Compose([transforms.Resize(32),
                                transforms.CenterCrop(32),
                                transforms.ToTensor()
                               ])# TODO: compose transforms here
dataset = datasets.ImageFolder(data_dir, transform=transform) # TODO: create the ImageFolder
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True) # TODO: use the ImageFolder dataset to create the DataLoader

<div dir=rtl>
در سلول بالا تمامی تصاویر دیتاست خوانده شده اند.

به دلیل اینکه تصاویر ابعاد گوناگونی دارند در این سلول با استفاده از تبدیل transform همگی تصاویر را به ابعاد 32 در 32 کاهش بعد داده ام.

سپس دیتالودرم را با استفاده از تصاویری که خوانده شده اند و ابعاد آنها تغییر یافته است در batch های 32تایی قرار داده ام و با فعال کردن درهم ریزی، ترتیب قرار گیری تصاویر از دو کلاس را بصورت رندوم تغییر داده ام.
</div>

In [ ]:
train_set_size = int(len(dataloader) * 0.8)
test_set_size = len(dataloader) - train_set_size
train_set, test_set = data.random_split(dataloader, [train_set_size, test_set_size])

<div dir=rtl>
از آنجایی که در متن تکلیف گفته شده است 80 درصد تصاویر را برای آموزش و 20 درصد تصاویر را برای تست مدل اختصاص دهیم، 

در سلول بالا ابتدا اندازه دو دسته داده های آموزشی و تست را مشخص کرده ام 

و سپس با استفاده از تابع random_split دیتالودر اصلی را به دو دیتالودر برای آموزش و تست افراز کرده ام که هر یک به تعدادی که ما قبلا مشخص کرده ایم تصویر دارند.
</div>

In [ ]:
def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

<div dir=rtl>
در تابع بالا یک تنسور را برای نمایش بصورت یک تصویر آماده میکنیم

سپس بررسی میکنیم اگر بنابر نرمال سازی تصویر بود آنرا با کمک وکتورهای میانگین و واریانس نرمال میکند،

سپس تصویر نهایی را نمایش میدهد.
</div>

In [ ]:
# Run this to test your data loader
images, labels = next(iter(dataloader))
imshow(images[0], normalize=False)

<div dir=rtl>
در اینجا یکی از تصاویر را بصورت رندوم نمایش میدهد.
</div>

In [ ]:
class Nerwork_noDropout_noBatchNormalization(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1),  # in_channels, out_channels, kernel_size, stride, padding
            nn.MaxPool2d(2),  # kernel_size
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 192, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 384, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True)
        )

        self.classifier = nn.Sequential(
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, output_dim),
        )

    def forward(self, x):
        x = self.features(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h

<div dir=rtl>
در سلول بالا معماری اصلی شبکه آمده است.

.

برای ساخت تمامی مدلهایی که در سوال یک و دو استفاده میشود از همین معماری اصلی استفاده میشود و تنها لایه ی Batch Normalization و Dropout به آن اضافه میشود.

این شبکه معماری ای شبیه به معماری شبکه alexnet دارد،

در این شبکه دو قسمت اصلی وجود دارد،
قسمت اول با هدف استخراج ویژگی از تصاویر نوشته شده است و قسمت دوم قصد دارد با استفاده از ویژگیهای استخراج شده تصاویر را دسته بندی کند.

.

در قسمت اول من از 5 لایه ی کانوولوشنی دوبعدی استفاده کردم،

بعد از هر لایه ی کانوولوشنی یک لایه ی maxpool دو بعدی قرار میگیرد و،

تابع فعالساز برای همه ی لایه ها تابع ReLU است

.

در قسمت دوم شبکه از سه لایه ی Linear و مجددا تابع فعالساز ReLU استفاده میکنیم.
</div>

In [ ]:
def initialize_parameters(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data, nonlinearity='relu')
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))
        nn.init.constant_(m.bias.data, 0)

<div dir=rtl>
در این تابع با استفاده از روش kaiming_normal و xavier اقدام به مقداردهی اولیه به پارامترهای مدل میکنیم.
</div>

## قسمت آ


In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def calculate_precision(y_pred, y):
    tp = (torch.max(y_pred, 1)[1].view(y.size()).data & y.data).sum()
    fp = (torch.max((torch.logical_not(y_pred)), 1)[1].view(y.size()).data & (torch.logical_not(y)).data).sum()
    precision = tp/(tp+fp)
    return precision

In [ ]:
def calculate_recall(y_pred, y):
    recall=metrics.recall_score((torch.max(y_pred, 1)[1]).cpu().numpy(), y.cpu().numpy())
    return recall

<div dir=rtl>
در سه سلول بالا پیاده سازی توابع محاسبه ی Accuracy، Precision و Recall از روی مقادیر پیشبینی شده و مقادیر اصلی قابل مشاهده است.
</div>

In [ ]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0
    epoch_prec = 0
    epoch_recall = 0
    
    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        y_pred, _ = model(x)
        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)
        prec = calculate_precision(y_pred, y)
        recall = calculate_recall(y_pred, y)
        loss.backward()

        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_prec += prec.item()
        epoch_recall += recall
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_prec / len(iterator), epoch_recall / len(iterator)

<div dir=rtl>
از این تابع برای پیاده سازی آموزش مدل استفاده میکنیم.

.

تابع به این صورت کار میکند که مدل، دیتالودر آموزش، تابع محاسبه loss مدل و نوع device در دسترس را دریافت نموده،

سپس تک تک داده های آموزش را به مدل نشان داده، 
مقدار پیشبینی شده را از مدل دریافت میکند،
با استفاده از مقدار پیشبینی شده و مقدار واقعی مقدار loss را محاسبه میکند،

سپس با استفاده از مقدار loss محاسبه شده جدید برای وزن ها و بایاس ها در گام loss.backward() محاسبه میشود،
 سپس با توجه به مقادیر جدید در گام optimizer.step() مدل optimize میشود.

 در هر گام سه معیار accuracy، precision و recall محاسبه میشود و نهایتا در آخر آموزش میانگین این سه مقدار به همراه میانگین loss بازگردانده میشود.
</div>

In [ ]:
def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0
    epoch_prec = 0
    epoch_recall = 0
    
    model.eval()

    with torch.no_grad():

#         for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):
        for x, y in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)
            prec = calculate_precision(y_pred, y)
            recall = calculate_recall(y_pred, y)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_prec += prec.item()
            epoch_recall += recall
           
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_prec / len(iterator), epoch_recall / len(iterator)

<div dir=rtl>
در این تابع قسمت ارزیابی مدل با استفاده از داده های تست پیاده سازی شده است.

.

در این قسمت همه ی داده های تست بصورت تک به تک به مدل نشان داده شده اند و از روی مقدار پیش بینی شده برای آنها مقدار loss، accuracy، precision و recall محاسبه میشود.

نهایتا میانگین این چهار مقدار به عنوان نتیجه ارزیابی مدل روی داده های تست بازگردانده میشود.
</div>

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

<div dir=rtl>
در این تابع با دریافت زمان شروع و زمان پایان فرآیند مقدار دقایق و ثانیه های صرق شده محاسبه میشود.
</div>

## قسمت ب

In [ ]:
OUTPUT_DIM = 2
model_noDropout_noBatch_adam = Nerwork_noDropout_noBatchNormalization(OUTPUT_DIM)
model_noDropout_noBatch_adam.apply(initialize_parameters)
model_noDropout_noBatch_adam = model_noDropout_noBatch_adam.to(device)
FOUND_LR = 1e-3
EPOCHS=5
optimizer = optim.Adam(model_noDropout_noBatch_adam.parameters(), lr=FOUND_LR)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

loss = []
acc = []
epochs = []
best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):

    start_time = time.monotonic()
    train_loss, train_acc, train_prec, train_recall = train(model_noDropout_noBatch_adam, train_set.dataset, optimizer, criterion, device)
    loss.append(train_loss)
    acc.append(train_acc)
    epochs.append(epoch)

    torch.save(model_noDropout_noBatch_adam.state_dict(), 'model_noDropout_noBatch_adam.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Prec: {train_prec*100:.2f}% | Train recall: {train_recall*100:.2f}%')

plt.plot(epochs, loss)
plt.plot(epochs, acc)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, acc, 'r', label='validation loss')
plt.title('Training loss and Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss and Acc')

<div dir=rtl>
در سلول بالا اولین مدل مبتنی بر شبکه Nerwork_noDropout_noBatchNormalization را آموزش داده ایم.

از تابع initialize_parameters برای مقداردهی اولیه به وزنها و بایاسهای شبکه استفاده شده است.

برای آموزش این مدل از تابع بهینه سازی Adam استفاده کرده ام،

فرآیند آموزش برای 5 ایپاک انجام شده است،

از تابع CrossEntropyLoss برای محاسبه loss مدل استفاده شده است،

نهایتا مدل، دیتالودر آموزش، optimizer و تابع محاسبه loss به تابع train داده شده است و این فرآیند به تعداد ایپاکها یعنی 5 بار تکرار شده است.

در پایان هر ایپاک مقدار loss، accuracy، precision و recall محاسبه شده اند و نمایش داده میشوند.

نهایتا همانطور که خواسته شده نمودار مقدار loss و accuracy فرآیند آموزش نمایش داده میشود.

.

در این فرآیند آموزش، از شبکه ای بدون Batch Normalization و Dropout و تابع بهینه سازی Adam استفاده کردم و مشاهده شده در پایان آموزش به دقت 94.34 درصد برای داده های آموزشی رسیدم.
</div>

In [ ]:
test_loss, test_acc, test_prec, test_recall = evaluate(model_noDropout_noBatch_adam, test_set.dataset, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test Precision: {test_prec:.3f} | Test Recall: {test_recall*100:.2f}%')

<div dir=rtl>
در سلول بالا فرآیند ارزیابی مدل روی داده های تست به وسیله ی تابع evaluate انجام شده است.

برای این بررسی 4 مقدار loss، accuracy، precision و recall نمایش داده شده است.

.
در این قسمت، از شبکه ای بدون Batch Normalization و Dropout و تابع بهینه ساز Adam استفاده کردم و مشاهده شده در دقت مدل برای داده های تست برابر با 94.65 درصد می باشد.

.

از آنجایی که دقت آموزش و تست هر دو حدود 94 درصد هستند نتیجه میگیریم شبکه، شبکه مناسبی بوده است و احتمال کمی وجود دارد که شبکه دچار overfit شده باشد.

</div>

In [ ]:
OUTPUT_DIM = 2
model_noDropout_noBatch_sgd = Nerwork_noDropout_noBatchNormalization(OUTPUT_DIM)
model_noDropout_noBatch_sgd.apply(initialize_parameters)
model_noDropout_noBatch_sgd = model_noDropout_noBatch_sgd.to(device)
FOUND_LR = 1e-3
EPOCHS=5
optimizer = optim.SGD(model_noDropout_noBatch_sgd.parameters(), lr=FOUND_LR, momentum=0.9)

loss = []
acc = []
epochs = []
best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):

    start_time = time.monotonic()
    train_loss, train_acc, train_prec, train_recall = train(model_noDropout_noBatch_sgd, train_set.dataset, optimizer, criterion, device)
    loss.append(train_loss)
    acc.append(train_acc)
    epochs.append(epoch)
    
    torch.save(model_noDropout_noBatch_sgd.state_dict(), 'model_noDropout_noBatch_sgd.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Prec: {train_prec*100:.2f}% | Train recall: {train_recall*100:.2f}%')

plt.plot(epochs, loss)
plt.plot(epochs, acc)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, acc, 'r', label='validation loss')
plt.title('Training loss and Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss and Acc')

<div dir=rtl>

در سلول بالا دومین مدل مبتنی بر شبکه Nerwork_noDropout_noBatchNormalization را آموزش داده ایم.

از تابع initialize_parameters برای مقداردهی اولیه به وزنها و بایاسهای شبکه استفاده شده است.

برای آموزش این مدل از تابع بهینه سازی SGD استفاده کرده ام،

فرآیند آموزش برای 5 ایپاک انجام شده است،

از تابع CrossEntropyLoss برای محاسبه loss مدل استفاده شده است،

نهایتا مدل، دیتالودر آموزش، optimizer و تابع محاسبه loss به تابع train داده شده است و این فرآیند به تعداد ایپاکها یعنی 5 بار تکرار شده است.

در پایان هر ایپاک مقدار loss، accuracy، precision و recall محاسبه شده اند و نمایش داده میشوند.

نهایتا همانطور که خواسته شده نمودار مقدار loss و accuracy فرآیند آموزش نمایش داده میشود.

.

در این فرآیند آموزش، از شبکه ای بدون Batch Normalization و Dropout و تابع بهینه سازی SGD استفاده کردم و مشاهده شده در پایان آموزش به دقت 93.78 درصد برای داده های آموزشی رسیدم.

</div>

In [ ]:
test_loss, test_acc, test_prec, test_recall = evaluate(model_noDropout_noBatch_sgd, test_set.dataset, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test Precision: {test_prec:.3f} | Test Recall: {test_recall*100:.2f}%')

<div dir=rtl>

در سلول بالا فرآیند ارزیابی مدل روی داده های تست به وسیله ی تابع evaluate انجام شده است.

برای این بررسی 4 مقدار loss، accuracy، precision و recall نمایش داده شده است.

.
در این قسمت، از شبکه ای بدون Batch Normalization و Dropout و تابع بهینه ساز SGD استفاده کردم و مشاهده شده در دقت مدل برای داده های تست برابر با 95.27 درصد می باشد.

.

از آنجایی که دقت آموزش حدود 93 درصد و دقت تست حدود 95 درصد هستند نتیجه میگیریم شبکه، شبکه مناسبی بوده است و احتمال کمی وجود دارد که شبکه دچار overfit شده باشد.

</div>

In [ ]:
OUTPUT_DIM = 2
model_noDropout_noBatch_rmsprop = Nerwork_noDropout_noBatchNormalization(OUTPUT_DIM)
model_noDropout_noBatch_rmsprop.apply(initialize_parameters)
model_noDropout_noBatch_rmsprop = model_noDropout_noBatch_rmsprop.to(device)
FOUND_LR = 1e-3
EPOCHS=5
optimizer = optim.RMSprop(model_noDropout_noBatch_rmsprop.parameters(), lr=FOUND_LR)

loss = []
acc = []
epochs = []
best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):

    start_time = time.monotonic()
    train_loss, train_acc, train_prec, train_recall = train(model_noDropout_noBatch_rmsprop, train_set.dataset, optimizer, criterion, device)
    loss.append(train_loss)
    acc.append(train_acc)
    epochs.append(epoch)

    torch.save(model_noDropout_noBatch_rmsprop.state_dict(), 'model_noDropout_noBatch_rmsprop.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Prec: {train_prec*100:.2f}% | Train recall: {train_recall*100:.2f}%')

plt.plot(epochs, loss)
plt.plot(epochs, acc)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, acc, 'r', label='validation loss')
plt.title('Training loss and Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss and Acc')

<div dir=rtl>

در سلول بالا سومین مدل مبتنی بر شبکه Nerwork_noDropout_noBatchNormalization را آموزش داده ایم.

از تابع initialize_parameters برای مقداردهی اولیه به وزنها و بایاسهای شبکه استفاده شده است.

برای آموزش این مدل از تابع بهینه سازی RMSProp استفاده کرده ام،

فرآیند آموزش برای 5 ایپاک انجام شده است،

از تابع CrossEntropyLoss برای محاسبه loss مدل استفاده شده است،

نهایتا مدل، دیتالودر آموزش، optimizer و تابع محاسبه loss به تابع train داده شده است و این فرآیند به تعداد ایپاکها یعنی 5 بار تکرار شده است.

در پایان هر ایپاک مقدار loss، accuracy، precision و recall محاسبه شده اند و نمایش داده میشوند.

نهایتا همانطور که خواسته شده نمودار مقدار loss و accuracy فرآیند آموزش نمایش داده میشود.

.

در این فرآیند آموزش، از شبکه ای بدون Batch Normalization و Dropout و تابع بهینه سازی RMSProp استفاده کردم و مشاهده شده در پایان آموزش به دقت 89.53 درصد برای داده های آموزشی رسیدم.

</div>

In [ ]:
test_loss, test_acc, test_prec, test_recall = evaluate(model_noDropout_noBatch_rmsprop, test_set.dataset, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test Precision: {test_prec:.3f} | Test Recall: {test_recall*100:.2f}%')

<div dir=rtl>

در سلول بالا فرآیند ارزیابی مدل روی داده های تست به وسیله ی تابع evaluate انجام شده است.

برای این بررسی 4 مقدار loss، accuracy، precision و recall نمایش داده شده است.

.
در این قسمت، از شبکه ای بدون Batch Normalization و Dropout و تابع بهینه ساز RMSProp استفاده کردم و مشاهده شده در دقت مدل برای داده های تست برابر با 92.54 درصد می باشد.

.

از آنجایی که دقت آموزش حدود 89 درصد و دقت تست حدود 92 درصد هستند نتیجه میگیریم شبکه، شبکه مناسبی بوده است و احتمال کمی وجود دارد که شبکه دچار overfit شده باشد.

</div>

## قسمت پ

<div dir=rtl>

تا کنون سه مدل آموزش داده ایم و عملکرد آنها را روی داده های آموزش و تست بررسی کرده ایم.

.

برای تمام مدل ها از شبکه Nerwork_noDropout_noBatchNormalization استفاده کرده ام.

* در اولین مدل از تابع بهینه ساز Adam استفاده کردم و به دقت 94.34 درصد برای داده های آموزشی و دقت 94.65 برای داده های تست رسیده ام

* در دومین مدل از تابع بهینه ساز SGD استفاده کردم و به دقت 93.78 برای داده های آموزشی و دقت 95.27 برای داده های تست رسیدم.

* در سومین مدل از تابع بهینه ساز RMSProp استفاده کردم و دقت مدل برای داده های آموزش به 89.53 و برای داده های تست برابر با 92.54 درصد شد.

.

از مقایسه نتایج این سه مدل نتیجه میگیریم دقت آموزش مدلی که با تابع بهینه ساز Adam آموزش دیده بود از همه ی مدلها بیشتر است و عملکرد مدلی که توسط بهینه ساز SGD آموزش دیده بود بهترین عملکرد را روی دسته بندی داده های تست دارد.


.

حال که دیدیم تابع بهینه ساز SGD برای شبکه ما بهترین نتیجه روی داده های تست را به ارمغان آورده، در ادامه این پروژه از همین تابع برای بهینه سازی مدل استفاده میکنم

</div>

## قسمت ت

In [ ]:
class Nerwork_Dropout_noBatchNormalization(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1),  # in_channels, out_channels, kernel_size, stride, padding
            nn.MaxPool2d(2),  # kernel_size
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 192, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 384, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, output_dim),
        )

    def forward(self, x):
        x = self.features(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h

<div dir=rtl>

شبکه مادری که در قسمت قبل معرفی کرده بودم را درنظربگیرید. 

در این قسمت تنها با اضافه کردن دو لایه Dropout در قسمت Classifier شبکه سعی میکنم فرآیند Regularization را انجام دهم.

پارامتر Dropout را در این شبکه برابر با 0.5 قرار داده ام.

</div>

In [ ]:
OUTPUT_DIM = 2
model_sgd_dropout = Nerwork_Dropout_noBatchNormalization(OUTPUT_DIM)
model_sgd_dropout.apply(initialize_parameters)
model_sgd_dropout = model_sgd_dropout.to(device)
FOUND_LR = 1e-3
EPOCHS=5
optimizer = optim.SGD(model_sgd_dropout.parameters(), lr=FOUND_LR, momentum=0.9)

loss = []
acc = []
epochs = []
best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):

    start_time = time.monotonic()
    train_loss, train_acc, train_prec, train_recall = train(model_sgd_dropout, train_set.dataset, optimizer, criterion, device)
    loss.append(train_loss)
    acc.append(train_acc)
    epochs.append(epoch)
    
    torch.save(model_sgd_dropout.state_dict(), 'model_sgd_dropout.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Prec: {train_prec*100:.2f}% | Train recall: {train_recall*100:.2f}%')

plt.plot(epochs, loss)
plt.plot(epochs, acc)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, acc, 'r', label='validation loss')
plt.title('Training loss and Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss and Acc')

<div dir=rtl>

در سلول بالا مدل مبتنی بر شبکه Nerwork_Dropout_noBatchNormalization را آموزش داده ایم.

از تابع initialize_parameters برای مقداردهی اولیه به وزنها و بایاسهای شبکه استفاده شده است.

برای آموزش این مدل از تابع بهینه سازی SGD استفاده کرده ام،

فرآیند آموزش برای 5 ایپاک انجام شده است،

از تابع CrossEntropyLoss برای محاسبه loss مدل استفاده شده است،

نهایتا مدل، دیتالودر آموزش، optimizer و تابع محاسبه loss به تابع train داده شده است و این فرآیند به تعداد ایپاکها یعنی 5 بار تکرار شده است.

در پایان هر ایپاک مقدار loss، accuracy، precision و recall محاسبه شده اند و نمایش داده میشوند.

نهایتا همانطور که خواسته شده نمودار مقدار loss و accuracy فرآیند آموزش نمایش داده میشود.

.

در این فرآیند آموزش، از شبکه ای با Dropout و بدون Batch Normalization و تابع بهینه سازی SGD استفاده کردم و مشاهده شده در پایان آموزش به دقت 91.31 درصد برای داده های آموزشی رسیدم.

</div>

In [ ]:
test_loss, test_acc, test_prec, test_recall = evaluate(model_sgd_dropout, test_set.dataset, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test Precision: {test_prec:.3f} | Test Recall: {test_recall*100:.2f}%')

<div dir=rtl>

در سلول بالا فرآیند ارزیابی مدل روی داده های تست به وسیله ی تابع evaluate انجام شده است.

برای این بررسی 4 مقدار loss، accuracy، precision و recall نمایش داده شده است.

.
در این قسمت، از شبکه ای با Dropout و  بدون Batch Normalization و تابع بهینه ساز SGD استفاده کردم و مشاهده شده در دقت مدل برای داده های تست برابر با 92.62 درصد می باشد.

.

از آنجایی که دقت آموزش حدود 91 درصد و دقت تست حدود 92 درصد هستند نتیجه میگیریم شبکه، شبکه مناسبی بوده است و احتمال کمی وجود دارد که شبکه دچار overfit شده باشد.

</div>

<div dir=rtl>

با مقایسه نتیجه مدل بدون لایه ی Dropout و مدل با لایه ی Dropout نتیجه میگیریم که افزودن این لایه منجر شده عملکرد مدل افت پیدا کند و از دقت 95.27 درصد برای داده های تست به 92.62 برای داده های تست برسد.

</div>

## قسمت س

In [ ]:
class Nerwork_Dropout_BatchNormalization(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1),  # in_channels, out_channels, kernel_size, stride, padding
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),  # kernel_size
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 192, 3, padding=1),
            nn.BatchNorm2d(192),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 384, 3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, output_dim),
        )

    def forward(self, x):
        x = self.features(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h

<div dir=rtl>

حال معماری شبکه قبلی که دارای لایه ی Dropout بود را در نظر بگیرید.

قصد داریم به همان شبکه لایه های Batch Normalization نیز اضافه کنیم،

برای این کار کافی است لایه های BatchNorm2d با ابعاد ورودی مناسب را بعد از لایه های conv2d اضافه کنیم.


</div>

In [ ]:
OUTPUT_DIM = 2
model_sgd_dropout_batch = Nerwork_Dropout_BatchNormalization(OUTPUT_DIM)
model_sgd_dropout_batch.apply(initialize_parameters)
model_sgd_dropout_batch = model_sgd_dropout_batch.to(device)
FOUND_LR = 1e-3
EPOCHS=5
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = optim.SGD(model_sgd_dropout_batch.parameters(), lr=FOUND_LR, momentum=0.9)

loss = []
acc = []
epochs = []
best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):

    start_time = time.monotonic()
    train_loss, train_acc, train_prec, train_recall = train(model_sgd_dropout_batch, train_set.dataset, optimizer, criterion, device)
    loss.append(train_loss)
    acc.append(train_acc)
    epochs.append(epoch)
    
    torch.save(model_sgd_dropout_batch.state_dict(), 'model_sgd_dropout_batch.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train Prec: {train_prec*100:.2f}% | Train recall: {train_recall*100:.2f}%')

plt.plot(epochs, loss)
plt.plot(epochs, acc)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, acc, 'r', label='validation loss')
plt.title('Training loss and Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss and Acc')

<div dir=rtl>

در سلول بالا مدل مبتنی بر شبکه Nerwork_Dropout_noBatchNormalization را آموزش داده ایم.

از تابع initialize_parameters برای مقداردهی اولیه به وزنها و بایاسهای شبکه استفاده شده است.

برای آموزش این مدل از تابع بهینه سازی SGD استفاده کرده ام،

فرآیند آموزش برای 5 ایپاک انجام شده است،

از تابع CrossEntropyLoss برای محاسبه loss مدل استفاده شده است،

نهایتا مدل، دیتالودر آموزش، optimizer و تابع محاسبه loss به تابع train داده شده است و این فرآیند به تعداد ایپاکها یعنی 5 بار تکرار شده است.

در پایان هر ایپاک مقدار loss، accuracy، precision و recall محاسبه شده اند و نمایش داده میشوند.

نهایتا همانطور که خواسته شده نمودار مقدار loss و accuracy فرآیند آموزش نمایش داده میشود.

.

در این فرآیند آموزش، از شبکه ای با Dropout و Batch Normalization و تابع بهینه سازی SGD استفاده کردم و مشاهده شده در پایان آموزش به دقت 93.86 درصد برای داده های آموزشی رسیدم.

</div>

In [ ]:
test_loss, test_acc, test_prec, test_recall = evaluate(model_sgd_dropout_batch, test_set.dataset, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test Precision: {test_prec:.3f} | Test Recall: {test_recall*100:.2f}%')

<div dir=rtl>

در سلول بالا فرآیند ارزیابی مدل روی داده های تست به وسیله ی تابع evaluate انجام شده است.

برای این بررسی 4 مقدار loss، accuracy، precision و recall نمایش داده شده است.

.
در این قسمت، از شبکه ای با Dropout و Batch Normalization و تابع بهینه ساز SGD استفاده کردم و مشاهده شده در دقت مدل برای داده های تست برابر با 97.85 درصد می باشد.

.

از آنجایی که دقت آموزش حدود 93 درصد و دقت تست حدود 98 درصد هستند نتیجه میگیریم شبکه، شبکه مناسبی بوده است و احتمال کمی وجود دارد که شبکه دچار overfit شده باشد.

</div>

## قسمت ج

<div dir=rtl>

حال در این قسمت سه مدلی که همگی با تابع بهینه ساز SGD آموزش دیده اند و تنها در ساختار شبکه با هم متفاوت بودند را بررسی میکنیم.

* مدل Nerwork_noDropout_noBatchNormalization در زمان آموزش به دقت 93.78 و در زمان تست به دقت 95.27 رسید.

* مدل Nerwork_Dropout_noBatchNormalization در زمان آموزش به دقت 91.31 و در زمان تست به دقت 92.62 رسید.

* مدل Nerwork_Dropout_BatchNormalization در زمان آموزش به دقت 93.86 و در زمان تست به دقت 97.85 رسید.

.

پس مشاهده میکنیم مدل آخر با وجود اینکه در زمان آموزش عملکردی متناسب با عملکرد اولین مدل داشته است ولی در زمان تست با اختلاف زیادی بهتر عمل کرده است و به وضوح میتوان تاثیر اضافه کردن لایه ی Dropout و Batch Normalization را در این قسمت مشاهده کرد.

</div>

## قسمت چ

<div dir=rtl>

در سلول های زیر توابعی که قبلا برای ساخت دیتالودر، ساخت شبکه و آموزش مدل نوشته بودیم را متناسب با نیاز ماژول wandb تغییر میدهیم

</div>

In [ ]:
def build_dataset(batch_size):
   
    # PATH = "C:\\uni\\1 - first term\\Deep Learning\\HW2 Practical"
    # data_dir = PATH + '\\train' # load from Kaggle
    data_dir = '/content/drive/MyDrive/data'
    transform = transforms.Compose([transforms.Resize(32),
                                    transforms.CenterCrop(32),
                                    transforms.ToTensor()
                                   ])# TODO: compose transforms here
    dataset = datasets.ImageFolder(data_dir, transform=transform) # TODO: create the ImageFolder
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True) # TODO: use the ImageFolder dataset to create the DataLoader

    train_set_size = int(len(dataloader) * 0.8)
    test_set_size = len(dataloader) - train_set_size
    train_set, test_set = data.random_split(dataloader, [train_set_size, test_set_size])
    return train_set, test_set

In [ ]:
class Nerwork_Dropout_BatchNormalization_build(nn.Module):
    def __init__(self, output_dim, dropout, batch_normalization):
        super().__init__()
        if batch_normalization == True:
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, 3, 2, 1),  # in_channels, out_channels, kernel_size, stride, padding
                nn.BatchNorm2d(64),
                nn.MaxPool2d(2),  # kernel_size
                nn.ReLU(inplace=True),
                nn.Conv2d(64, 192, 3, padding=1),
                nn.BatchNorm2d(192),
                nn.MaxPool2d(2),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 384, 3, padding=1),
                nn.BatchNorm2d(384),
                nn.ReLU(inplace=True),
                nn.Conv2d(384, 256, 3, padding=1),
                nn.BatchNorm2d(256),
                nn.ReLU(inplace=True),
                nn.Conv2d(256, 256, 3, padding=1),
                nn.BatchNorm2d(256),
                nn.MaxPool2d(2),
                nn.ReLU(inplace=True)
            )

            self.classifier = nn.Sequential(
                nn.Dropout(dropout),
                nn.Linear(256 * 2 * 2, 4096),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),
                nn.Linear(4096, 4096),
                nn.ReLU(inplace=True),
                nn.Linear(4096, output_dim),
            )
        else:
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, 3, 2, 1),  # in_channels, out_channels, kernel_size, stride, padding
                nn.MaxPool2d(2),  # kernel_size
                nn.ReLU(inplace=True),
                nn.Conv2d(64, 192, 3, padding=1),
                nn.MaxPool2d(2),
                nn.ReLU(inplace=True),
                nn.Conv2d(192, 384, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(384, 256, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(256, 256, 3, padding=1),
                nn.MaxPool2d(2),
                nn.ReLU(inplace=True)
            )

            self.classifier = nn.Sequential(
                nn.Dropout(dropout),
                nn.Linear(256 * 2 * 2, 4096),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),
                nn.Linear(4096, 4096),
                nn.ReLU(inplace=True),
                nn.Linear(4096, output_dim),
            )
            
    def forward(self, x):
        x = self.features(x)
        h = x.view(x.shape[0], -1)
        x = self.classifier(h)
        return x, h

In [ ]:
def build_network(OUTPUT_DIM, dropout, batch_normalization):
    network = Nerwork_Dropout_BatchNormalization_build(OUTPUT_DIM, dropout, batch_normalization)
    return network.to(device)

In [ ]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    
    elif optimizer == "rmsprop":
        optimizer = optim.RMSprop(network.parameters(), 
                                  lr=learning_rate)
    return optimizer

<div dir=rtl>

این تابع برای استفاده از توابع بهینه ساز گوناگون نوشته شده است.

</div>

In [ ]:
def train_epoch(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0
    epoch_prec = 0
    epoch_recall = 0
    
    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)
        prec = calculate_precision(y_pred, y)
        recall = calculate_recall(y_pred, y)
        loss.backward()

        optimizer.step()
        wandb.log({"batch loss": loss.item()})
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_prec += prec.item()
        epoch_recall += recall
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_prec / len(iterator), epoch_recall / len(iterator)

In [ ]:
OUTPUT_DIM = 2
def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        train_set, test_set = build_dataset(config.batch_size)
        network = build_network(OUTPUT_DIM, config.dropout, config.batch_normalization)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        criterion = nn.CrossEntropyLoss()
        criterion = criterion.to(device)
        for epoch in trange(config.epochs, desc="Epochs"):
            train_loss, train_acc, train_prec, train_recall = train_epoch(network, train_set.dataset, optimizer, criterion, device)
            wandb.log({"loss": train_loss, "epoch": epoch})           

<div dir=rtl>

در سلول بالا فرآیند آموزش را به گونه ای تغییر داده ایم که متناسب با نیاز wandb باشد و بتواند برای حالاتی که از اون میخواهیم مدلهای گوناگون و مناسبی آموزش دهد.

</div>

In [ ]:
!pip install wandb -Uq

In [ ]:
# !pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'dropout': {
          'values': [0.2, 0.3, 0.4, 0.5, 0.6]
        },
    'batch_normalization': {
            'values': [True, False]
        }
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'values': [1, 3 , 5, 7]}
    })


parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

<div dir=rtl>

در سلول بالا تنظیمات گوناگونی که من میخواهم بهترین حالات از بین آنها انتخاب شود را مشخص کرده ام.

</div>

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="pytorch_sweeps_HW2_5")

In [ ]:
wandb.agent(sweep_id, train, count=5)

<div dir=rtl>

در سلول بالا با استفاده از ماژول wandb برای کانفیگ مشخص شده فرآیند آموزش متوالی مدل انجام شده است.

نتیجه ی اجرای این سلول را میتوانید در لینک زیر مشاهده نمایید:

https://wandb.ai/farkoo/pytorch_sweeps_HW2_5/sweeps/lxhy9tdi?workspace=user-farzanehkoohestani1999

</div>

<div dir=rtl>

همانطور که در لینک بالا قابل مشاهده است در این بخش بهترین تنظمیات شبکه که این ابزار مشخص کرده است را گزارش میدهم:

</div>

* Batch Normalization: True
* Batch Size: 5
* Dropout: 0.2
* Epochs: 3
* Learning Rate: 0.008
* Optimizer: SGD



# سوال 2

In [ ]:
def classify(model, image_transforms, image):
  model = model.eval()
  image = image_transforms(image).float()
  image = image.to(device)
  image = image.unsqueeze(0)
  
  output = model(image)
  _, predicted = torch.max(output[0], 1)

  return predicted.item()

<div dir=rtl>

در این تابع با دریافت مدل، تبدیل کننده ی تصویر و خود تصویر، خروجی اجرای مدل روی تصویر به دست آمده و برگردانده میشود.

</div>

In [ ]:
MAIN = '/content/drive/MyDrive/_test_'
cascPath = '/content/drive/MyDrive/haarcascade_frontalface_default.xml'
label = [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1]
mask_dict = {
  0: "With Mask",
  1: "Without Mask",
}
acc = 0
for i in range(1, 21):
  imagePath = os.path.join(MAIN, str(i) + '.jpeg')
  faceCascade = cv2.CascadeClassifier(cascPath)
  image = Image.open(imagePath)

  # Detect faces in the image
  faces = faceCascade.detectMultiScale(
      np.array(image),
      scaleFactor=1.1,
      minNeighbors=5,
      minSize=(30, 30),
      flags = cv2.CASCADE_SCALE_IMAGE
  )
  for (x, y, w, h) in faces:
    face = image.crop((x, y, x+w, y+h))
  predicted = classify(model_sgd_dropout_batch, transform, face)
  display(face)
  print('Predicted:', mask_dict[predicted], '    *****    ', 'Actual:', mask_dict[label[i-1]])
  if predicted == label[i-1]:
        acc += 1
acc = acc/20
print('\n \n ------------------------------------ \n Test Accuracy: ', acc)

<div dir=rtl>

در این قسمت 20 تصویر، که 10 تای آن با ماسک و 10 تای آن بدون ماسک است استفاده شده است.

تک به تک این تصاویر خوانده شده اند و با استفاده از کد آماده ای در opencv ناحیه ی صورت تشخیص داده و برش شده است.

سپس این تصویر برش داده شده به تابع Classify که قبلا به آن پرداختیم به همراه تبدیل transform که ابعاد تصویر را به 32 در 32 تبدیل میکرد به همراه بهترین مدلی که تا کنون آموزش داده ایم یعنی model_sgd_dropout_batch داده شده است.

سپس تصویر برش داده شده، مقدار پیش بینی شده برای کلاس آن و کلاس واقعی تصویر را نمایش میدهد.

.

برای تک تک تصاویر این مراحل طی میشود و دقت مدل روی این 20 تصویر محاسبه و نمایش داده میشود.

مدل ما 19 تصویر از 20 تصویر را درست دسته بندی کرده است و دقت نهایی آن برابر با 95 درصد بدست آمده است.

</div>

# سوال 3

<div dir=rtl>

در این قسمت از مدل resnet50 برای بهره گیری از یادگیری انتقالی استفاده میکنیم.

</div>

In [ ]:
from torchvision import datasets, models, transforms

model = models.resnet50(pretrained=True)
for param in model.parameters():
  param.requires_grad = False
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Linear(num_features, 2) #(num_of_class == 2)
model = model.to(device) 

<div dir=rtl>

در سلول بالا مدل resnet50 دانلود میشود سپس،

سپس با flase کردن param.requires_grad مشخص میکنیم که قصد نداریم لایه های درونی مدل آموزش ببیند و از وزنهایی که قبلا بدست آمده است میخواهیم استفاده کنیم.

سپس با اضافه کردن یک لایه Linear خروجی آخرین لایه را به تعداد کلاسهایمان یعنی 2 کاهش میدهیم.

مدل resnet50 برای آموزش آماده است.

</div>

In [ ]:
def train_(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    epoch_prec = 0
    epoch_recall = 0
    model.train()
    for (x, y) in tqdm(iterator, desc="Training", leave=False):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        _, y_pred = torch.max(outputs, 1)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        acc = metrics.accuracy_score(y_pred.cpu().numpy(), y.cpu().numpy())#calculate_accuracy(y_pred, y)
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

<div dir=rtl>

از این تابع برای آموزش مدل pretrain شده استفاده میکنیم.

</div>

In [ ]:
model_resnet50 = model
OUTPUT_DIM = 2
FOUND_LR = 1e-3
EPOCHS=5
optimizer = optim.SGD(model_resnet50.parameters(), lr=FOUND_LR, momentum=0.9)

loss = []
acc = []
epochs = []
best_valid_loss = float('inf')

for epoch in trange(EPOCHS, desc="Epochs"):

    start_time = time.monotonic()
    train_loss, train_acc = train_(model_resnet50, train_set.dataset, optimizer, criterion, device)
    loss.append(train_loss)
    acc.append(train_acc)
    epochs.append(epoch)
    
    torch.save(model_resnet50.state_dict(), 'model_resnet50.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

plt.plot(epochs, loss)
plt.plot(epochs, acc)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, acc, 'r', label='validation loss')
plt.title('Training loss and Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss and Acc')

<div dir=rtl>

در سلول بالا مدل pretrain شده ی resnet50 را آموزش داده ایم.

برای آموزش این مدل از تابع بهینه سازی SGD استفاده کرده ام،

فرآیند آموزش برای 5 ایپاک انجام شده است،

از تابع CrossEntropyLoss برای محاسبه loss مدل استفاده شده است،

نهایتا مدل، دیتالودر آموزش، optimizer و تابع محاسبه loss به تابع train داده شده است و این فرآیند به تعداد ایپاکها یعنی 5 بار تکرار شده است.

در پایان هر ایپاک مقدار loss و accuracy محاسبه شده اند و نمایش داده میشوند.

نهایتا همانطور که خواسته شده نمودار مقدار loss و accuracy فرآیند آموزش نمایش داده میشود.

.

نهایتا مشاهده میکنیم دقت آموزش برای این مدل برابر با 80 درصد میشود.

</div>

In [ ]:
def classify_(model, image_transforms, image):
  model = model.eval()
  image = image_transforms(image).float()
  image = image.to(device)
  image = image.unsqueeze(0)
  output = model(image)
  _, predicted = torch.max(output, 1)
  return predicted.item()

In [ ]:
MAIN = '/content/drive/MyDrive/_test_'
cascPath = '/content/drive/MyDrive/haarcascade_frontalface_default.xml'
label = [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1]
mask_dict = {
  0: "With Mask",
  1: "Without Mask",
}
acc = 0
for i in range(1, 21):
  imagePath = os.path.join(MAIN, str(i) + '.jpeg')
  faceCascade = cv2.CascadeClassifier(cascPath)
  image = Image.open(imagePath)

  # Detect faces in the image
  faces = faceCascade.detectMultiScale(
      np.array(image),
      scaleFactor=1.1,
      minNeighbors=5,
      minSize=(30, 30),
      flags = cv2.CASCADE_SCALE_IMAGE
  )
  for (x, y, w, h) in faces:
    face = image.crop((x, y, x+w, y+h))
  predicted = classify_(model_resnet50, transform, face)
  display(face)
  print('Predicted:', mask_dict[predicted], '    *****    ', 'Actual:', mask_dict[label[i-1]])
  if predicted == label[i-1]:
        acc += 1
acc = acc/20
print('\n \n ------------------------------------ \n Test Accuracy: ', acc)

<div dir=rtl>

در این قسمت 20 تصویر، که 10 تای آن با ماسک و 10 تای آن بدون ماسک است استفاده شده است.

تک به تک این تصاویر خوانده شده اند و با استفاده از کد آماده ای در opencv ناحیه ی صورت تشخیص داده و برش شده است.

سپس این تصویر برش داده شده به تابع Classify که قبلا به آن پرداختیم به همراه تبدیل transform که ابعاد تصویر را به 32 در 32 تبدیل میکرد به همراه مدل resnet50 داده شده است.

سپس تصویر برش داده شده، مقدار پیش بینی شده برای کلاس آن و کلاس واقعی تصویر را نمایش میدهد.

.

برای تک تک تصاویر این مراحل طی میشود و دقت مدل روی این 20 تصویر محاسبه و نمایش داده میشود.

مدل resnet50 سیزده تصویر از 20 تصویر را درست دسته بندی کرده است و دقت نهایی آن برابر با 65 درصد بدست آمده است.

</div>